## Identify and plot Geophone 3 and Geophone 4 events

### Import libraries

In [1]:
import pandas as pd
from obspy import read,UTCDateTime
from datetime import datetime, timedelta
import numpy as np
import os
import glob
import sys
import matplotlib.pyplot as plt
from PIL import Image

# Import functions
fxndir = '../functions/'
sys.path.insert(0,fxndir)
from moon2data import *

### Inputs to obtain waveforms

In [2]:
parentdir = '/data/ytamama/Apollo17/LSPE_data/sac_volts_ds/'
minfreq = 3
maxfreq = 35
befwin = 15
aftwin = 30

### Load moonquake catalogs

In [3]:
# Catalog with information by geophone
mqdir = '../catalogs/final_catalogs/'
cat1 = pd.read_csv(mqdir + 'A17_moonquakes_catalog_HQ_final.csv')
cat1.drop(list(cat1.filter(regex='Unnamed|index')), axis=1, inplace=True)

# Catalog with information averaged per event
cat2 = pd.read_csv(mqdir + 'A17_moonquakes_catalog_HQ_avg_event_stats.csv')
cat2.drop(list(cat2.filter(regex='Unnamed|index')), axis=1, inplace=True)

### Distinguish between isolated and repeating

In [4]:
# Repeating
cat1_rpt = cat1.loc[cat1.isol_or_rpt == 'repeating'].reset_index()
cat2_rpt = cat2.loc[cat2.isol_or_rpt == 'repeating'].reset_index()
evid_refs = np.unique(cat2_rpt.evid_ref.tolist())

# Isolated
cat1_isol = cat1.loc[cat1.isol_or_rpt == 'isolated'].reset_index()
cat2_isol = cat2.loc[cat2.isol_or_rpt == 'isolated'].reset_index()
evids_isol = np.unique(cat2_isol.evid.tolist())

## 1. Identify possible Geophone 3 and 4 events

### Identify families whose:

1. Stacked waveforms have the largest signal at Geophone 3 and whose average emergence at Geophone 3 is under 5s
2. Stacked waveforms have the largest signal at Geophone 4 and whose average emergence at Geophone 4 is under 5s

In [5]:
evid_refs_geo3 = []
evid_refs_geo4 = []

# Iterate through families
for evid_ref in evid_refs:
    
    # Data for each event in family
    rows1 = cat1_rpt.loc[cat1_rpt.evid_ref == evid_ref]
    rows2 = cat2_rpt.loc[cat2_rpt.evid_ref == evid_ref]
    evids_family = np.unique(rows2.evid.tolist())
    num_evts = len(evids_family)

    # Stack waveforms
    firstst1 = 1
    firstst2 = 1
    firstst3 = 1
    firstst4 = 1
    for evid in evids_family:
        row2 = rows2.loc[rows2.evid == evid].iloc[0]
        arrtime = datetime.strptime(row2.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')

        # Geophone 1
        st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
        if (st1 != '') & (firstst1 == 1):
            evtst1 = st1
            firstst1 = 0
        elif (st1 != '') & (firstst1 == 0):
            evtst1 += st1

        # Geophone 2
        st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
        if (st2 != '') & (firstst2 == 1):
            evtst2 = st2
            firstst2 = 0
        elif (st2 != '') & (firstst2 == 0):
            evtst2 += st2

        # Geophone 3
        st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
        if (st3 != '') & (firstst3 == 1):
            evtst3 = st3
            firstst3 = 0
        elif (st3 != '') & (firstst3 == 0):
            evtst3 += st3

        # Geophone 4
        st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
        if (st4 != '') & (firstst4 == 1):
            evtst4 = st4
            firstst4 = 0
        elif (st4 != '') & (firstst4 == 0):
            evtst4 += st4
            
    # Stack
    # Geophone 1
    st_stack1 = evtst1.copy()
    st_stack1.stack(npts_tol=2)
    trdata1 = st_stack1.traces[0].data
    # Geophone 2
    st_stack2 = evtst2.copy()
    st_stack2.stack(npts_tol=2)
    trdata2 = st_stack2.traces[0].data
    # Geophone 3
    st_stack3 = evtst3.copy()
    st_stack3.stack(npts_tol=2)
    trdata3 = st_stack3.traces[0].data
    # Geophone 4
    st_stack4 = evtst4.copy()
    st_stack4.stack(npts_tol=2)
    trdata4 = st_stack4.traces[0].data

    # Check on which geophone is the amplitude highest
    pgv_geo1 = np.max(np.abs(trdata1))
    pgv_geo2 = np.max(np.abs(trdata2))
    pgv_geo3 = np.max(np.abs(trdata3))
    pgv_geo4 = np.max(np.abs(trdata4))
    df = pd.DataFrame(data = {'geophones':[1, 2, 3, 4], 'PGV':np.array([pgv_geo1, pgv_geo2, pgv_geo3, pgv_geo4])})
    df = df.sort_values(by=['PGV'],ascending=False)
    geophones = df.geophones.tolist()
   
    # If amplitudes highest at Geophone 3
    if (geophones[0] == 3):
        # Check emergences 
        rows1_geo3 = rows1.loc[rows1.geophone == 3]
        if len(rows1_geo3) > 0:
            emergence_mean = np.mean(rows1_geo3.emergence_s.tolist())
            if emergence_mean <= 5:
                evid_refs_geo3.append(evid_ref)
            
    # If amplitudes highest at Geophone 4
    elif (geophones[0] == 4):
        # Check emergences 
        rows1_geo4 = rows1.loc[rows1.geophone == 4]
        if len(rows1_geo4) > 0:
            emergence_mean = np.mean(rows1_geo4.emergence_s.tolist())
            if emergence_mean <= 5:
                evid_refs_geo4.append(evid_ref)


### Repeat for isolated events

In [6]:
evids_geo3_isol = []
evids_geo4_isol = []

# Iterate through isolated events
for evid in evids_isol:
    rows1 = cat1_isol.loc[cat1_isol.evid == evid]
    row2 = cat2_isol.loc[cat2_isol.evid == evid].iloc[0]
    arrtime = datetime.strptime(row2.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')

    # Traces at each geophone
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    
    # Check on which geophone is the amplitude highest
    pgv_geo1 = np.max(np.abs(trdata1))
    pgv_geo2 = np.max(np.abs(trdata2))
    pgv_geo3 = np.max(np.abs(trdata3))
    pgv_geo4 = np.max(np.abs(trdata4))
    df = pd.DataFrame(data = {'geophones':[1, 2, 3, 4], 'PGV':np.array([pgv_geo1, pgv_geo2, pgv_geo3, pgv_geo4])})
    df = df.sort_values(by=['PGV'],ascending=False)
    geophones = df.geophones.tolist()
    
    # Amplitude greatest at Geophone 3
    if (geophones[0] == 3):
        # Check emergence at Geophone 3
        row_geo3 = cat1_isol.loc[(cat1_isol.evid == evid) & (cat1_isol.geophone == 3)]
        if len(row_geo3) > 0:
            row_geo3 = row_geo3.iloc[0]
            emergence_geo3 = np.mean(row_geo3.emergence_s.tolist())
            if emergence_geo3 <= 5:
                evids_geo3_isol.append(evid)
    
    # Amplitude greatest at Geophone 4
    elif (geophones[0] == 4):
        # Check emergence at Geophone 4
        row_geo4 = cat1_isol.loc[(cat1_isol.evid == evid) & (cat1_isol.geophone == 4)]
        if len(row_geo4) > 0:
            row_geo4 = row_geo4.iloc[0]
            emergence_geo4 = np.mean(row_geo4.emergence_s.tolist())
            if emergence_geo4 <= 5:
                evids_geo4_isol.append(evid)


## 2. Plot candidate Geophone 3, Geophone 4 events

Determined from visual inspection

### Repeating Geophone 3 events

In [7]:
cat1_rpt_geo3 = cat1[cat1['evid_ref'].isin(evid_refs_geo3)]
evids_geo3 = np.unique(cat1_rpt_geo3.evid.tolist())
savedir = '/data/ytamama/figures/Apollo17/geo3_events/repeating/'
pngnames = []

# Obtain waveforms
for evid in evids_geo3:
    row = cat2.loc[cat2.evid == evid].iloc[0]
    evid_ref = row.evid_ref
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0')
    ax.plot(trtimes2,trdata2_norm+4, color='C1')
    ax.plot(trtimes3,trdata3_norm+2, color='C2')
    ax.plot(trtimes4,trdata4_norm, color='C3')
    ax.set_title(f'EVID {evid}, (REF: {evid_ref})',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'EVID' + evid + '_waveforms.png')
    plt.close()

In [8]:
# Aggregate figures into PDF
images = [
    Image.open(f)
    for f in pngnames
]
geo34_dir = './geo3_geo4_waveforms/'
pdf_path = geo34_dir + 'possible_geo3_repeating.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)

### Isolated Geophone 3 events

In [9]:
cat1_isol_geo3 = cat1[cat1['evid'].isin(evids_geo3_isol)]
savedir = '/data/ytamama/figures/Apollo17/geo3_events/isolated/'
pngnames = []

# Obtain waveforms
for evid in evids_geo3_isol:
    row = cat2.loc[cat2.evid == evid].iloc[0]
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0')
    ax.plot(trtimes2,trdata2_norm+4, color='C1')
    ax.plot(trtimes3,trdata3_norm+2, color='C2')
    ax.plot(trtimes4,trdata4_norm, color='C3')
    ax.set_title(f'EVID {evid}',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'EVID' + evid + '_waveforms.png')
    plt.close()

In [10]:
# Aggregate figures into PDF
images = [
    Image.open(f)
    for f in pngnames
]
pdf_path = geo34_dir + 'possible_geo3_isolated.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)

### Repeating Geophone 4 events

In [11]:
cat1_rpt_geo4 = cat1[cat1['evid_ref'].isin(evid_refs_geo4)]
evids_geo4 = np.unique(cat1_rpt_geo4.evid.tolist())
savedir = '/data/ytamama/figures/Apollo17/geo4_events/repeating/'
pngnames = []

# Obtain waveforms
for evid in evids_geo4:
    row = cat2.loc[cat2.evid == evid].iloc[0]
    evid_ref = row.evid_ref
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0')
    ax.plot(trtimes2,trdata2_norm+4, color='C1')
    ax.plot(trtimes3,trdata3_norm+2, color='C2')
    ax.plot(trtimes4,trdata4_norm, color='C3')
    ax.set_title(f'EVID {evid}, (REF: {evid_ref})',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'EVID' + evid + '_waveforms.png')
    plt.close()

In [12]:
# Aggregate figures into PDF
images = [
    Image.open(f)
    for f in pngnames
]
pdf_path = geo34_dir + 'possible_geo4_repeating.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)

### Isolated Geophone 4 events

In [13]:
cat1_isol_geo4 = cat1[cat1['evid'].isin(evids_geo4_isol)]
savedir = '/data/ytamama/figures/Apollo17/geo4_events/isolated/'
pngnames = []

# Obtain waveforms
for evid in evids_geo4_isol:
    row = cat2.loc[cat2.evid == evid].iloc[0]
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0')
    ax.plot(trtimes2,trdata2_norm+4, color='C1')
    ax.plot(trtimes3,trdata3_norm+2, color='C2')
    ax.plot(trtimes4,trdata4_norm, color='C3')
    ax.set_title(f'EVID {evid}',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'EVID' + evid + '_waveforms.png')
    plt.close()

In [14]:
# Aggregate figures into PDF
images = [
    Image.open(f)
    for f in pngnames
]
pdf_path = geo34_dir + 'possible_geo4_isolated.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)